Run IDF in Parallel
===================

This tutorial will guide you on how to use a Python script to run EnergyPlus simulations using IDF files in parallel. By leveraging multiple CPU cores, parallel execution can significantly decrease simulation time for large batches of IDF files.

Prerequisites:
--------------

-   EnergyPlus installed and added to your system's PATH.
-   Python 3.x installed.
-   Required Python packages: `eppy`, `bs4`, and `pandas`. Install them using `pip install eppy bs4 pandas`.

Steps:
------

1.  Prepare your IDF Files: Store all your IDF files that you want to simulate in a directory, named `idf` by default.

2.  Weather File: Ensure you have an EPW weather file available. This tutorial uses 'USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw' as the default.

3.  IDD File: Make sure the EnergyPlus IDD file corresponding to the IDF version you're using is available. The default in this script is "Energy+_22_2_0.idd".

4.  Running the Script: Run the script using Python. The function `lazy_run_IDF()` is invoked by default and initiates the parallel simulations.

5.  Output: Simulation results will be saved in directories under the `results` directory, with each simulation result being in a folder named `results_filename`, where `filename` is the name of the corresponding IDF file.

6.  Parsing Results: If you want to extract specific results, the functions available will parse the `.htm` output files and extract values such as district heating demand and total building area.

FAQ:
----

Q: How do I change the number of CPUs used for the simulation?

A: In the `lazy_run_IDF()` function, modify the `num_CPUs` parameter. Set it to the desired number of CPUs or to "all" to use all available CPUs.

Q: How do I change the directory where my IDF files are located?

A: Adjust the `idf_dir` parameter in the `lazy_run_IDF()` function.

Q: My simulation timestep is different. How do I modify it?

A: Change the `simulation_timestep` parameter in the `lazy_run_IDF()` function to your desired value.

Q: I get an error related to the IDD version. What should I do?

A: Make sure that the version of the IDD file you're pointing to in the `iddfile` parameter matches the version of the IDF files you're trying to simulate.

Q: Can I extract different values from the simulation results?

A: Yes, you can modify the `extract_heating_value()` function or create a new function to extract different values from the `.htm` result files.

Q: My simulation isn't writing results to the correct directory. How do I fix this?

A: Adjust the `results_dir` parameter in the `lazy_run_IDF()` function to your desired directory.

# Enter E+ version here

Make sure you have the same version of energy plus installed as your IDF file

In [1]:
path_to_energyplus_folder = r'C:\EnergyPlusV22-2-0'

### Step 1 - Some things to do before running the script

In [4]:
from eppy_parallel_helper import *
energy_plus_path = path_to_energyplus_folder + r'\energyplus.exe'
path_to_idd = path_to_energyplus_folder + r'\Energy+.idd'
path_to_weather_file = path_to_energyplus_folder + r'\WeatherData\USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw'
# Set the path to the EnergyPlus executable
os.environ['PATH'] += os.pathsep + energy_plus_path



### Step 2 - Lazy run in parallel (without loading all into memory)

In [5]:
lazy_run_IDF(
    num_CPUs = 'all',                                               #Change to "all" to use all available CPUs
    iddfile = path_to_idd,                                          #This is the path to the Energy+.idd file found inside C:\EnergyPlusV22-2-0
    epwfile = path_to_weather_file,                                 #This is a weather data file inside C:\EnergyPlusV22-2-0\WeatherData
    idf_dir = 'idf',                                                #This is the directory where the idf files are located
    results_dir = 'results',                                        #This is the directory where the results will be saved
    simulation_timestep = 6)                                        #This is the simulation timestep per hour in minutes

### Step 3 - Post-process results

In [3]:
result_dir = 'results'
parse_results(result_dir)

Results saved to results.csv


,idf_name,district_heating_demand_kwh,total_building_area_m2,normalised_district_heating_demand_kwh_m2
0,a57460b41497,93761.66,2785.91,33.655667
1,a57460b55824,106539.90,3387.65,31.449500
2,a71787b41497,111294.69,3129.76,35.560136
3,a71787b55824,124075.45,3731.50,33.250824


# Detailed overview

The script is designed to automate EnergyPlus simulations for multiple IDF (input data file) models in parallel. It utilizes the `eppy` library for interacting with IDF files and running EnergyPlus simulations.

### Summary:

- **Simulation Settings Changed**: The timestep is changed to 1, and the unit conversion for output tables is set to `JtoKWH`. These are found in the `modified_idf_run_generator` function.

- **Variables Fetched**: The script extracts district heating demand, total building area, and normalised district heating demand from the simulation result HTML files. The exact lines in the HTML file where this information resides is hard-coded, which is why only the first 200 lines are processed for performance reasons (as defined in the `extract_heating_value` function). If the format of the result files changes, or if additional/other information needs to be extracted, this function will need to be updated.


#### Import Necessary Libraries

```python

import os

import glob

import pandas as pd

from bs4 import BeautifulSoup

from eppy.modeleditor import IDF

from eppy.runner.run_functions import runIDFs

```

- **os, glob**: These are standard Python libraries for interacting with the system, directories, and files.

- **pandas**: Data analysis library.

- **BeautifulSoup**: Library for web scraping purposes to pull the data out of HTML files.

- **eppy**: A Python library for scripting EnergyPlus.

#### IDF Configuration and Running

##### 1. Create EPLaunch Options

```python

def make_eplaunch_options(idf, output_dir):

```

This function creates necessary options for the `runIDFs` function, which determines how the simulation will be executed.

- **idf**: The IDF object we're processing.

- **output_dir**: Desired directory for the simulation results.

Inside the function:

1\. Extract the EnergyPlus version from the IDF.

2\. Build a set of options which includes EnergyPlus version, output prefix (name of the IDF file without extension), output suffix, output directory, and settings to read variables and expand objects.

##### 2. Generate List of IDF Files

```python

def idf_file_generator(idf_dir):

```

This function is a generator that yields the paths to IDF files in a specified directory.

- **idf_dir**: The directory containing the IDF files.

##### 3. Generate Modified IDFs for Simulation

```python

def modified_idf_run_generator(epwfile, idf_dir):

```

This function generates IDF objects and their respective run options. It modifies certain simulation settings of each IDF.

- **epwfile**: Weather file for the simulation.

- **idf_dir**: Directory containing the IDF files.

Inside the function, it modifies:

1\. Simulation timestep to 1 (`Number_of_Timesteps_per_Hour`).

2\. Energy conversion units to `JtoKWH` for output tables (`OUTPUTCONTROL:TABLE:STYLE`).

These settings can be changed by modifying these lines.

##### 4. Main Execution for Running IDF Files

```python

def lazy_run_IDF( ... ):

```

This function orchestrates the parallel execution of EnergyPlus simulations.

It sets the required IDD (input data dictionary) version, modifies the IDF properties (like simulation timestep), and finally runs the simulations in parallel.

Inside this function:

1\. It changes the `TIMESTEP` of the IDF files.

2\. Modifies the output directory for each IDF file based on the `results_dir` parameter.

3\. Finally, runs the simulations in parallel using `runIDFs`.

#### Result Parsing and Extraction

##### 5. Extract Heating Value

```python

def extract_heating_value(htm_file, num_lines=200):

```

This function extracts district heating demand and the total building area from an EnergyPlus result HTML file.

- **htm_file**: Path to the HTML file containing the results.

- **num_lines**: For performance reasons, only the first 200 lines of the HTML file are read. This value can be modified to read more or fewer lines.

Inside, it uses BeautifulSoup to parse the HTML tables and extract the desired values. If you need different values, you'll need to change how this function parses the HTML tables.

##### 6. Get All HTML Files from a Directory

```python

def get_htm_files(results_dir):

```

This function fetches all `.htm` result files from a given directory.

##### 7. Extract and Process Values

```python

def get_values(htm_files, verbose=False):

```

This function goes through each HTML results file and extracts values using the `extract_heating_value` function. The extracted values are district heating demand, total building area, and normalized heating demand.

##### 8. Parse Simulation Results

```python

def parse_results(results_dir):

```

This function uses the above functions to extract results, process them into a pandas DataFrame, and save that DataFrame to a CSV file.

